#K Nearest Neighbours

KNN is a simple classification algorithm. It classifies cases based on a similarity measure relying on the labels belonging to the K nearest points in the training set.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import neighbors as neigh
from sklearn.cross_validation import train_test_split
import operator
import matplotlib.pyplot as plt
from sklearn import feature_selection
from sklearn import preprocessing

%matplotlib inline  


Let's define all the functions we need to implement KNN classification algorithm.

Similarity: Calculate the distance between two data instances.

####Define Euclidean Distance: 

In [ ]:
def euclideanDistance(instance1, instance2):
    length = len(instance1)
    # you can also check if instance1 and instance2 have the same length
    distance = 0
    for l in range(length):
        distance += (instance1[l] - instance2[l])**2
    return math.sqrt(distance)

For example:

In [ ]:
data1 = [0,1,2]
data2 = [0,2,4]
distance = euclideanDistance(data1, data2)
print 'Distance: ', distance

Let's define a function to get the K nearest neighbors of a point in a set

In [ ]:
def getNeighbors(data, labels, testInstance, K):
    distances = []
    neighbors = {}
    #Finds the distances between all the points and creates a list of tuples.
    for i in range(len(data)):
        dist = euclideanDistance(testInstance, data[i, :])
        distances.append([data[i,:], dist])

    #Sorts the list of distances by using the second element of the tuple, i.e. the distance    

    idx = np.argsort(np.array(distances)[:, 1])
    neighbors_data = data[idx]
    neighbors_label = labels[idx]
    
    neighbors =  {'data': neighbors_data[:K], 'labels': neighbors_label[:K]}
    return neighbors
    

For example:

In [ ]:
# define the training set: 2 points and 2 labels
data = np.array([[2, 2, 2], [4, 4, 4]])
labels = np.array([0, 1])

# define the test instance
testInstance = [5, 5, 5]

# choose the number of neighbours
K = 1

# find & retrieve the K nearest points to the test instance, sorted by the distance
neighbors = getNeighbors(data, labels, testInstance, K)
print(neighbors)

Response: count the number of times a certain class appears in the set of neighbours. The class with the highest frequency will be the label assigned to the test instance.

In [ ]:
def getResponse(neighbors):
    classVotes = {}
    #Assign the votes for every class
    for i in range(len(neighbors)):
        response = neighbors[i]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    
    #Use the dictionary to short which class has the most votes
    sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
    #print sortedVotes
    return sortedVotes[0][0]

For example:

In [ ]:
# in this case we have two 1s and one 0: class 1 wins.

neighbors['labels'] = np.array([1, 1, 0])
response = getResponse(neighbors['labels'])
print(response)

Accuracy: test the performance of the model.

In [ ]:
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        #If the label of the testSet and the prediction are the same add one.
        if testSet[i] == predictions[i]:
            correct += 1
    return (float(correct)/float(len(testSet))) * 100.0

For example:

In [ ]:
# true labels
testSet = np.array(['a','a','b'])

# predicted labels
predictions = ['a', 'a', 'a']

accuracy = getAccuracy(testSet, predictions)
print(accuracy)

**Exercise:** <br> Assign a label to the test instance, basing on the following training set:

In [ ]:
training_set = np.array([[1, 1, 1], [1, 3, 5], [7, 5, 4], [9, 5, 3]])
training_labels = np.array([1, 2, 1, 2])
test_instance = np.array([4, 4, 4])

# get K neighbours
K = 1
neighbours = getNeighbors(training_set, training_labels, testInstance, K)

# get the label
label = getResponse(neighbours['labels'])

print label

# what about the accuracy?

### And now, with real data

At this point, we have all the tools we need to classify data. Now we want to test the algorithm over real data, namely the Statlog (German Credit Data) Data Set (http://bit.ly/1K3bcku). Each customer is described by a set of numbers (features), and we want to decide automatically whether he or she is a "good" or a "bad" customer. This means we are in a binary classification setup. 

First, we want to read and explore our data:

In [ ]:
data=pd.read_csv('german.csv',header=None)

In [ ]:
data.describe()

To test the algorithm, we need to split the data into training and test set, and convert to Numpy array.

In [ ]:
train, test = train_test_split(data, train_size = 0.7)

train_X = np.array(train)[:, :24]
train_Y = np.array(train)[:,24]

test_X = np.array(test)[:, :24]
test_Y = np.array(test)[:,24]


1 - NN algorithm:

In [ ]:
predictions=[]
K = 1
for i in range(len(test_Y)):
    neighbors = getNeighbors(train_X, train_Y, test_X[i,:], K)
    result = getResponse(neighbors['labels'])
    predictions.append(result)
accuracy = getAccuracy(test_Y, predictions)
print 'Accuracy: ', accuracy, '%'

We can use sklearn library

In [ ]:
clf = neigh.KNeighborsClassifier(K)
clf.fit(train_X, train_Y)

predictions1 = clf.predict(test_X)

accuracy = getAccuracy(test_Y, predictions1)
print('Accuracy: ' + repr(accuracy) + '%')

###Normalizing data

If we have a look at the values in the data, we can see that they have different orders of magnitude for different features. A normalization step might be required.

In [ ]:
data.head()

In [ ]:
# compute mean and standard deviation of training set
mean = np.mean(train_X, axis=0)
std = np.std(train_X, axis=0)

# note that we scale test set using the mean and std of the training set
train_Xscaled = (train_X-mean)/std
test_Xscaled = (test_X-mean)/std


In [ ]:
K = 1
clf = neigh.KNeighborsClassifier(K)
clf.fit(train_Xscaled, train_Y)

predictions = clf.predict(test_Xscaled)

accuracy = getAccuracy(test_Y, predictions)
print('Accuracy: ' + repr(accuracy) + '%')

We can also investigate other metrics, such as: 

In [ ]:
target_names= ['good', 'bad']
print(sklearn.metrics.classification_report(test_Y, predictions, target_names=target_names))

We can also try setting weights to see if our performance increases

In [ ]:
K = 1
clf = neigh.KNeighborsClassifier(K, weights='distance')
clf.fit(train_X, train_Y)

predictions = clf.predict(test_X)

# accuracy = getAccuracy(test_Y, predictions)
accuracy = sklearn.metrics.accuracy_score(test_Y, predictions)*100
print('Accuracy: ' + repr(accuracy) + '%')

What happens if we increase the  number of neighbours taken into account? We can plot the accuracy accordingly.

In [ ]:
def plotvector(train_X, train_Y, test_X, test_Y, weights, upperLim = 100):
    results = []
    for k in range(1, upperLim, 4):
        clf = neigh.KNeighborsClassifier(n_neighbors = k, weights = weights)
        clf = clf.fit(train_X, train_Y)
        preds = clf.predict(test_X)
        accuracy = clf.score(test_X, test_Y)
        results.append([k, accuracy*100])
 
    results = np.array(results)
    return(results)

pltvector1 = plotvector(train_X, train_Y, test_X, test_Y, weights = "uniform")
pltvector2 = plotvector(train_X, train_Y, test_X, test_Y,  weights = "distance")
line1 = plt.plot(pltvector1[:,0], pltvector1[:,1], label = "uniform")
line2 = plt.plot(pltvector2[:,0], pltvector2[:,1], label = "distance")
plt.legend(loc=3)
plt.ylim(60, 80)
plt.title("Accuracy with Increasing K")
plt.show()

We can also do a step of feature selection, in order to maintain only the most descriptive features. More specifically, the sklearn.feature_selection module can be used for feature selection/dimensionality reduction on sample sets, either to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets. Univariate feature selection works by selecting the best features based on univariate statistical tests.

First, we select the optimal number of features, through cross validation:

In [ ]:
percentiles = range(1, 100, 5)
results = []
for i in range(1, 100, 5):
    fs = feature_selection.SelectPercentile(sklearn.feature_selection.f_classif, percentile=i)
    X_train_fs = fs.fit_transform(train_Xscaled, train_Y)
    scores = sklearn.cross_validation.cross_val_score(clf, X_train_fs, train_Y, cv=5)
    results = np.append(results, scores.mean())

optimal_percentil = np.where(results == results.max())[0]
print "Optimal percentil :{0}".format(percentiles[optimal_percentil]), "\n"

# Plot number of features VS. cross-validation scores
import pylab as pl
pl.figure()
pl.xlabel("Number of features selected")
pl.ylabel("Cross validation accuracy)")
pl.plot(percentiles,results)
print "Mean scores:",results


Then, we select the relevant features and we repeat the KNN algorithm with the transformed data:

In [ ]:
fs = sklearn.feature_selection.SelectPercentile(sklearn.feature_selection.f_classif, percentile=percentiles[optimal_percentil])
X_train_fs = fs.fit_transform(train_Xscaled, train_Y)

clf = sklearn.neighbors.KNeighborsClassifier(5)

clf.fit(X_train_fs, train_Y)
X_test_fs = fs.transform(test_Xscaled)
predictions = clf.predict(X_test_fs)

accuracy = getAccuracy(test_Y, predictions)
print('Accuracy: ' + repr(accuracy) + '%')


What happens to accuracy if we change the ratio between training and test set?
